In [1]:
import pandas as pd
import wave
import contextlib

In [ ]:
def countfiles(path, filetype):
    # show how many video files
    allfiles = os.listdir(rootpath)
    files = [ fname for fname in allfiles if fname.endswith(filetype)]
    # files = [ fname for fname in allfiles if fname.endswith('.mp4')]
    print(files[0:5])
    print(len(files))
    return files

### Silence duration time

1. labeled pariticpants silence response time by Aduacity
2. calculate silence duration time (s) = start - end by google sheet.

In [ ]:
# normalised silence duration time
df_silence_A1_help =  pd.read_csv('/home/nali/Develop/HRI/data/HRI_Study2_result_csv/silence_duration/part2/condition_A1_withhelp/condition_A1_help.csv')
# df_silence_B =  pd.read_csv('/home/nali/Develop/HRI/silentduration_B.csv')

In [ ]:
#normalise
min_silence= df_silence_A1_help['silent_length'].min()
max_silence= df_silence_A1_help['silent_length'].max()
range_max_min = max_silence - min_silence
print(min_silence, max_silence)
df_silence_A1_help['norm_silentlength'] = (df_silence_A1_help['silent_length']-min_silence) / range_max_min

In [ ]:
df_silence_A1_help.to_csv('/home/nali/Develop/HRI/data/HRI_Study2_result_csv/silence_duration/part2/condition_A1_withhelp/condition_A1_help.csv')

### Only Participants speech extraction
1. Labelled participants speech by Aduactiy
2. Cut audio chips from labelled files (.txt)
3. Concatenate audio chips into one partipcant audio file

In [ ]:
audiopath = '/media/nali/Seagate Portable Drive/6_HRIStudy2_audioData/part2/condition_A2_B/'

In [ ]:
configfile_path = '/media/nali/Seagate Portable Drive/HRI_Study2_result_csv/participantSpeech/Part2_A2_B/'

In [ ]:
audio_files = countfiles(audiopath, '.mp3')
config_files = countfiles(configfile_path, '.txt')

In [ ]:
# step 2
index = 0
for audio in audio_files:
    audio_name = audio.split('.')[0]
    audio_config = configfile_path + audio_name + '.txt'
# Inicialize the extractor
    ext = AudioClipExtractor( audiopath + audio, '/usr/bin/ffmpeg')
# Extract the clips according to the specs and save them as a zip archive
    if(os.path.exists(audio_config)):
        ext.extract_clips(audio_config, configfile_path, zip_output=True)
        index +=1
    else:
        print(audio_config)
print("done!!", index)

In [ ]:
#  folder name: config_files + clips
# step 3
# extract zip files of participant audios 
# concatenate these audio files into one audio for each zipped folder
index = 0
error = 0
for config in config_files:
    foldername = config.split('.')[0] + '_clips'
    auidofiles = '/media/nali/Seagate Portable Drive/8_HRIStudy2_audioData_participantonly/part2_A2_B/' + foldername + '/'
    # auidofiles = '/home/nali/Develop/HRI/data/HRI_Study2_result_csv/participantSpeech/79_face_A24_BA2_trim_clips/'
    allfiles = os.listdir(auidofiles)
    allfiles.sort()
    try:
        files = [ AudioFileClip(auidofiles + fname) for fname in allfiles if fname.endswith('.mp3')]
        final_audio = concatenate_audioclips(files)
        final_audio.write_audiofile('/media/nali/Seagate Portable Drive/8_HRIStudy2_audioData_participantonly/part2_A2_B/mp3/'
                                + config.split('.')[0] + '_user' +'.mp3')
    except:
        print("****error****", foldername)
        error +=1
    index += 1

print("done", index, error)

### Audio Signal Feature Extraction

#### 1. MFCCs - Mel Frequency Cepstral Coefficients
scipy

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import pandas as pd

from scipy.io import wavfile as sciwav
from scipy import signal

In [ ]:
def load_audiofiles(path):
    all_audio_files = os.listdir(path)
    audio_files = [ fname for fname in all_audio_files if fname.endswith('.wav')]
    audio_files.sort()
    print(audio_files[:10])
    return audio_files

In [ ]:
audiopath = '/media/nali/Seagate Portable Drive/8_HRIStudy2_audioData_participantonly/part2_A1_beforehelp/wav/'
audiofiles = load_audiofiles(audiopath)

In [ ]:
#get MMCCs features
signal, fs = librosa.load(audiopath + audiofiles[1])
s_len = len(signal)

if s_len < mean_signal_length:
    pad_len = mean_signal_length - s_len
    pad_rem = pad_len % 2
    pad_len //= 2
    signal = np.pad(signal, (pad_len, pad_len + pad_rem), 'constant', constant_values = 0)
else:
    pad_len = s_len - mean_signal_length
    pad_len //= 2
    signal = signal[pad_len:pad_len + mean_signal_length]
mfcc = librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=39)
mfcc = mfcc.T
feature = mfcc

In [ ]:
mfcc_feature = []
mfcc_feature.append([mfcc, audiofiles[1]])

In [ ]:
# mfcc_feature_all = {}
mfcc_feature_all =  pd.DataFrame(mfcc_feature, columns = ['feature','filename'])
mfcc_feature_all.to_csv("the path you like.csv")

#### 2. Pitch

pip install AMFM-decompy

YAAPT: yet another algorithm for pitch tracking

In [4]:
from pydub import AudioSegment
import amfm_decompy.pYAAPT as pYAAPT
import amfm_decompy.basic_tools as basic
import matplotlib.pyplot as plt

In [ ]:
def updateMono(path, name):
    sound = AudioSegment.from_wav(path)
    sound = sound.set_channels(1)
    sound.export(output_mono+name, format="wav")

In [ ]:
output_mono = '/media/nali/Seagate Portable Drive/8_HRIStudy2_audioData_participantonly/part2_A1_B/wav_mono/'
folderpath = '/media/nali/Seagate Portable Drive/8_HRIStudy2_audioData_participantonly/part2_A1_B/wav/'
allfiles = load_audiofiles(folderpath)

In [ ]:
index = 0
for file in allfiles:
    name = file.split('.')[0] + '_mono.wav'
    updateMono(folderpath + file, name)
    if (index%10 == 0):
        print("mono processing", index)
    index+=1
    
print("done", index)  

In [ ]:
df_pitchValues = pd.DataFrame(columns=['file', 'pitch', 'median', 'mean', 'q1', 'q3'])
df_pitchValues

In [ ]:
def pichvalue(pathfile, index):
    signal_new = basic.SignalObj(pathfile)
    # print(pathfile)
    pitchYaapt_new = pYAAPT.yaapt(signal_new)
    df_pitchValues.loc[index] = [pathfile.split('/')[-1], pitchYaapt_new.samp_values,
                                 np.median(pitchYaapt_new.samp_values), np.mean(pitchYaapt_new.samp_values),
                                 np.percentile(pitchYaapt_new.samp_values, 25, interpolation = 'midpoint'),
                                 np.percentile(pitchYaapt_new.samp_values, 75, interpolation = 'midpoint')]

In [ ]:
index = 0
for file in load_audiofiles(output_mono):
    
    try:
        pichvalue(output_mono + file, index)
    except:
        print(output_mono+ file, index)
        
    if (index%10 == 0):
        print("pitch processing", index)
    index +=1
    
print("done", index) 

In [ ]:
df_pitchValues

In [ ]:
median = np.median(df_pitchValues.iloc[0]['pitch'])

In [ ]:
dp_pitch = pd.DataFrame(df_pitchValues['pitch'], columns=['pitch'])
dp_pitch.head()

In [ ]:
#to_csv
df_pitchValues.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/participantSpeech/pitch/part2_A1_B.csv")